In [4]:
import cv2
import numpy as np

def find_object(template_path, search_path, min_matching_points=5):
    # Read the images
    template_img = cv2.imread(template_path)
    search_img = cv2.imread(search_path)

    # Check if the images are loaded successfully
    if template_img is None or search_img is None:
        print(f"Error: Unable to read images at {template_path} and {search_path}")
        return None

    # Convert images to grayscale
    template_gray = cv2.cvtColor(template_img, cv2.COLOR_BGR2GRAY)
    search_gray = cv2.cvtColor(search_img, cv2.COLOR_BGR2GRAY)

    # Initialize SIFT detector
    sift = cv2.SIFT_create()

    # Detect keypoints and compute descriptors
    keypoints_template, descriptors_template = sift.detectAndCompute(template_gray, None)
    keypoints_search, descriptors_search = sift.detectAndCompute(search_gray, None)

    # Initialize a Brute Force Matcher
    bf = cv2.BFMatcher()

    # Match descriptors
    matches = bf.knnMatch(descriptors_template, descriptors_search, k=2)

    # Apply ratio test
    good_matches = [m1 for m1, m2 in matches if m1.distance < 0.75 * m2.distance]
    matched_points = [keypoints_template[m.queryIdx].pt for m in good_matches]

    # Draw bounding box around the found object in the search image
    if len(good_matches) > 4:
        src_pts = np.float32([keypoints_template[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
        dst_pts = np.float32([keypoints_search[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)

        M, _ = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
        h, w = template_gray.shape

        # Decrease bounding box height from the top
        corners = np.float32([[0, h * 0.06], [0, h - 1], [w - 1, h - 1], [w - 1, h * 0.06]]).reshape(-1, 1, 2)
        transformed_corners = cv2.perspectiveTransform(corners, M)

        # Draw bounding box with decreased height from the top
        search_img = cv2.polylines(search_img, [np.int32(transformed_corners)], True, (0, 0, 255), 2, cv2.LINE_AA)

    # Check if the number of matching points is enough
    if len(good_matches) >= min_matching_points:
        identification_message = "The object can be identified in this image."

        # Display the matches, bounding box, and identification message
        draw_params = dict(matchColor=(0, 255, 0), singlePointColor=None, matchesMask=None, flags=2)
        img_result = cv2.drawMatches(template_img, keypoints_template, search_img, keypoints_search, good_matches, None, **draw_params)
        text_position = (img_result.shape[1] // 2 - 100, img_result.shape[0] - 10)  # Bottom middle
        cv2.putText(img_result, identification_message, text_position, cv2.FONT_HERSHEY_SIMPLEX, 0.8, (112, 112, 112), 2)
        cv2.imshow("Matches and Bounding Box", img_result)

    else:
        print("CANNOT find the target object.")
        identification_message = "CANNOT find the target object."
        text_position = (10, 30)  # Top left
        cv2.putText(search_img, identification_message, text_position, cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
        cv2.imshow("Search Image", search_img)

    cv2.waitKey(0)
    cv2.destroyAllWindows()

    return matched_points

# Example usage with different image paths
template_path = r'target.png'
search_path = r'test2.jpg'
matched_coordinates = find_object(template_path, search_path)

# Print the coordinates of matched points
if matched_coordinates is not None:
    print("Matched Coordinates:", matched_coordinates)


CANNOT find the target object.
Matched Coordinates: [(83.3129653930664, 132.7753143310547)]
